# Qwen2.5 Quick API Test & Data Loading

This notebook demonstrates basic Qwen2.5 API usage and dataset loading for the seminar.  
**For full evaluation and reproducible experiments, use the dedicated task-specific notebooks:**
- Chest X-ray: Classification, Grounding
- Brain MRI: Description, Detection, Diagnosis

In [2]:
import os
from openai import OpenAI

from dotenv import load_dotenv
# Try loading both .env and user.env for compatibility
load_dotenv(dotenv_path="../config/user.env")

api_key = os.environ.get("OPENROUTER_API_KEY")
if not api_key:
    print("WARNING: OPENROUTER_API_KEY is not set or not loaded from env file.")

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=api_key,
)

In [7]:
import base64

# Function to get base64 string from a local file
def encode_image_to_data_uri(path: str) -> str:
    with open(path, "rb") as f:
        b64 = base64.b64encode(f.read()).decode('utf-8')
    return f"data:image/png;base64,{b64}"

# Use local PNG file
data_uri = encode_image_to_data_uri("VLM-Seminar25-Dataset/chest_xrays/images/0cbaca12e05803e6301f8f4a92b47565.png")
 
completion = client.chat.completions.create(

    model="qwen/qwen2.5-vl-72b-instruct:free",
    messages=[
        {
            "role": "user",
            "content": [
            {"type": "text", "text": "Given the medical image, classify it as 'healthy' or 'unhealthy'. It is very important that you only output only either 'healthy' or 'unhealthy'."},  
            {"type": "image_url", "image_url": {"url": data_uri}},
            ],
        }
    ],
)
print(completion.choices[0].message.content)

The image is classified as 'unhealthy'. The X-ray shows a chest that has undergone surgery, indicated by the presence of surgical clips and possibly a drain. These features suggest a recent intervention, which typically follows from an unhealthy condition that required treatment. Additionally, the texture and appearance of the surrounding tissues and organs may also show signs of illness or recovery that would not be present in a healthy chest X-ray.


In [ ]:
# test load data 
import json
import os
import random
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import numpy as np

# ====== Configuration ======
FOLDER = 'VLM-Seminar25-Dataset/chest_xrays'
ANNOTATIONS_PATH = os.path.join(FOLDER, 'annotations_len_50.json')
IMAGES_PATH = os.path.join(FOLDER, 'images')
FONT_PATH = "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf"  # fallback to default if not found
FONT_SIZE = 32
FIG_SIZE = (10, 5)
BASE_COLORS = ['#ff9999','#99ccff','#99ff99','#ffcc99','#cc99ff','#99ffff','#ff99ff','#ffff99']

# ====== Helper Functions ======
def draw_text(draw, pos, text, font, color, outline=2):
    for dx in [-outline, outline]:
        for dy in [-outline, outline]:
            draw.text((pos[0]+dx, pos[1]+dy), text, font=font, fill='black')
    draw.text(pos, text, font=font, fill=color)

def get_color(label, color_map):
    if label not in color_map:
        if len(color_map) < len(BASE_COLORS):
            color_map[label] = BASE_COLORS[len(color_map)]
        else:
            color_map[label] = tuple(np.random.rand(3,))
    return color_map[label]

def visualize_sample(img_id, ann, color_map):
    img_file = os.path.join(IMAGES_PATH, img_id + '.png')
    if not os.path.exists(img_file):
        print(f"Image not found: {img_file}")
        return None
    
    img = Image.open(img_file).convert("RGB")
    draw = ImageDraw.Draw(img)

    try:
        font = ImageFont.truetype(FONT_PATH, FONT_SIZE)
    except:
        font = ImageFont.load_default()

    for x1, y1, x2, y2, label in ann.get('bbox_2d', []):
        color = get_color(label, color_map)
        for w in range(6):
            draw.rectangle([x1-w, y1-w, x2+w, y2+w], outline=color)
        bbox_text = draw.textbbox((0,0), label, font=font)
        text_w = bbox_text[2] - bbox_text[0]
        text_h = bbox_text[3] - bbox_text[1]
        text_pos = (x1, y2 + 10) if y1 < 50 else (x1, y1 - text_h - 10)
        draw_text(draw, text_pos, label, font, color)

    return img

# ====== Main Execution ======
with open(ANNOTATIONS_PATH) as f:
    annotations = json.load(f)

healthy_keys = [k for k,v in annotations.items() if v['status'] == 'healthy']
unhealthy_keys = [k for k,v in annotations.items() if v['status'] == 'unhealthy']

sample_keys = []
if healthy_keys:
    sample_keys.append(random.choice(healthy_keys))
if unhealthy_keys:
    sample_keys.append(random.choice(unhealthy_keys))

color_map = {}

fig, axes = plt.subplots(1, len(sample_keys), figsize=FIG_SIZE)
if len(sample_keys) == 1:
    axes = [axes]

for i, key in enumerate(sample_keys):
    ann = annotations[key]
    img = visualize_sample(key, ann, color_map)
    if img:
        axes[i].imshow(img)
        gd = ann.get('global_disease')
        gd_text = ', '.join(gd) if gd else 'null'
        axes[i].set_title(f"ID: {key}\nStatus: {ann['status']}\nGlobal Disease: {gd_text}", fontsize=16)
        axes[i].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
text = None
image_url = None

In [2]:
completion = client.chat.completions.create(
  extra_headers={
    "HTTP-Referer": "<YOUR_SITE_URL>", # Optional. Site URL for rankings on openrouter.ai.
    "X-Title": "<YOUR_SITE_NAME>", # Optional. Site title for rankings on openrouter.ai.
  },
  extra_body={},
  model="qwen/qwen2.5-vl-72b-instruct:free",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "What is in this image?"
        },
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
          }
        }
      ]
    }
  ]
)
print(completion.choices[0].message.content)

The image depicts a serene natural landscape with a wooden boardwalk path cutting through a lush green field. The boardwalk leads the viewer's eye towards a line of trees in the distance under a bright blue sky dotted with scattered clouds. The vibrant green grass and the clear sky suggest a pleasant, sunny day, creating a peaceful and inviting scene.
